
<br>
-------------------------------------------------------------------------------<br>
Name:        energeticEvaluationsMultiOutputRegression.py<br>
Purpose:     Evaluation of energetic impact of HVAC system based on multi-output regression model using the static data set<br>
Author:      Marcus Vogt<br>
Created:     10.12.2021<br>
Copyright:   Chair of Sustainable Manufacturing and Life Cycle Engineering, Institute of Machine Tools and Production Technology, Technische UniversitÃ¤t Braunschweig, Langer Kamp 19b, 38106 Braunschweig, Germany<br>
Licence:     MIT (see License)<br>
-------------------------------------------------------------------------------<br>


In [15]:
import energeticEvaluationsMultiOutputRegression as enEvalMulti
import os
import pandas as pd
import numpy as np
import helpers.helperFuncs as helperFuncs

In [18]:
########### read in data set ############
current_dir = os.getcwd()
staticResultPath = os.path.join(current_dir, "data", "automateSimulationStaticResults.csv")
dfStaticResults = pd.read_csv(staticResultPath, index_col=0)
dfStaticResults = helperFuncs.extendStaticDF(dfStaticResults)

########### boundary parameters ############
averageOutsideRelativeHumidity = 77.7820776255707  # Todo: Note: With the current best models outside humidity not even needed
averageOutsideTemperatureDegrees = 10.7060730593607
useBoundaryParameters = False
if useBoundaryParameters:
  maxHumansInAirFlow = 3  # maximum number of humans in air flow zones in room
  maxHumansInRoom = 6  # total maximum amount of humans in room
  maxWasteHeatRoomW = 4000  # max waste heat in W
  roomDewPointDegrees = -50  # dew point temperature in °C
  inletDewPointDegrees = -60  # dew point temperature in °C
  inletTemperatureDegrees = 20  # inlet temperature in °C
  leakagesSuctionVolumeFlowM3H = 500  # volume flow in m^3/h
else: # this approach can be used if the boundary conditions below are known
  scalingFactorS = 1.1 # scaling factor calculated from necessary air flow
  maxMoistureLoad = 0.5 # max. moisture load in kg/s
  maxWasteHeatRoomW = 3000  # max waste heat in W

In [17]:
########### evaluations ############
if useBoundaryParameters:
  scalingFactorS, maxMoistureLoad = enEvalMulti.boundaryParameters2scalingFactorMoistureLoad(maxHumansInAirFlow=maxHumansInAirFlow,
                                                                             maxHumansInRoom=maxHumansInRoom,
                                                                             roomDewPointDegrees=roomDewPointDegrees,
                                                                             inletDewPointDegrees=inletDewPointDegrees,
                                                                             inletTemperatureDegrees=inletTemperatureDegrees,
                                                                             leakagesSuctionVolumeFlowM3H=leakagesSuctionVolumeFlowM3H)

print("The scaling factor S is: {}, volume flow is: {} m^3/h, max. moisture load is: {} kg/s"
    .format(round(scalingFactorS, 3), round(scalingFactorS*11000, 3), round(maxMoistureLoad, 7)))

d, n_scores = enEvalMulti.interpolateScalingFactorAndInternalLoadsMultiOutputRegression(dfStaticResults,
                            averageOutsideRelativeHumidity=averageOutsideRelativeHumidity,
                            averageOutsideTemperature=averageOutsideTemperatureDegrees, scalingFactorS=scalingFactorS,
                            maxWasteHeatRoomW=maxWasteHeatRoomW, maxMoistureLoad=maxMoistureLoad)
print("The final results of the interpolation based on multi-output regression model are:")
print(d)
# summarize performance
print("Mean Absolute Error: %.3f kWh. Standard deviation: (%.3f) kWh" % (np.mean(n_scores), np.std(n_scores)))



The scaling factor S is: 1.1, volume flow is: 12100.0 m^3/h, max. moisture load is: 0.5 kg/s
The final results of the interpolation based on multi-output regression model are:
{'electricEnergyKwh': 433947.65625, 'naturalGasEnergyKwh': 521295.78125, 'districtHeatingEnergyKwh': 152219.5625}
Mean Absolute Error: 16421.670 kWh. Standard deviation: (2597.470) kWh
